## Prepare the data for the model ##

The processed data used in the paper is available on the github repo. However, detailed information on steps required to process the data is not. These pre-processing notebooks aim to create a dataset that can be easily used by the model.

We take the following steps:

<img src="../../images/data_path.png" alt="Data Path" style="width:60%;">

Demean, detrend, and resample are done in the [Data_Cleaning](Data_Cleaning.ipynb) notebook.



Read each file and segment them into one hour windows. Each file will contain 28,800 samples per hour at an 8 Hz sampling rate.

In [1]:
import os
from obspy import read
import glob
import numpy as np
from obspy import UTCDateTime

# Define the processed and output folder paths
processed_folder = os.getcwd() + '/data/processed'
segmented_output_folder = os.getcwd() + '/data/segmented'
os.makedirs(segmented_output_folder, exist_ok=True)  # Create output directory if it doesn't exist

# Define the target segment duration (1 hour) in seconds
segment_duration = 3600  # seconds

# Process each processed file for segmentation
for file_path in glob.glob(f"{processed_folder}/*.mseed"):
    # Read the processed file
    st = read(file_path)
    
    # Normalize the traces in the stream object - done in data_cleaning
    #st.normalize()
    
    # Split each trace in the Stream object into one-hour segments
    for tr in st:
        start_time = tr.stats.starttime
        end_time = tr.stats.endtime
        segment_start = start_time
        
        # Loop over each one-hour segment
        while segment_start + segment_duration <= end_time:
            # Define the end time for the current segment
            segment_end = segment_start + segment_duration

            # Slice the trace to create a one-hour segment
            segment = tr.slice(starttime=segment_start, endtime=segment_end)
            
            # Format the filename for the segment
            segment_filename = f"{tr.stats.network}_{tr.stats.station}_{tr.stats.channel}_{segment_start.strftime('%Y%m%dT%H%M%S')}.mseed"
            segment_filepath = os.path.join(segmented_output_folder, segment_filename)
            
            # Save the one-hour segment as a new file
            try:
                segment.write(segment_filepath, format="MSEED")
                print(f"Saved segment file: {segment_filename}")
            except Exception as e:
                print(f"Error writing segment {segment_filename}: {e}")

            # Move to the next hour
            segment_start = segment_end

            # # Save the one-hour segment as a new file
            # segment.write(segment_filepath, format="MSEED")
            # print(f'Saved segment file: {segment_filepath}')

            # # Move to the next hour
            # segment_start = segment_end

Saved segment file: HV_STCD_HHE_20241019T224800.mseed
Saved segment file: HV_STCD_HHE_20241019T234800.mseed
Saved segment file: HV_STCD_HHE_20241020T004800.mseed
Saved segment file: HV_STCD_HHE_20241020T014800.mseed
Saved segment file: HV_STCD_HHE_20241020T024800.mseed
Saved segment file: HV_STCD_HHE_20241020T034800.mseed
Saved segment file: HV_STCD_HHE_20241020T044800.mseed
Saved segment file: HV_STCD_HHE_20241020T054800.mseed
Saved segment file: HV_STCD_HHE_20241020T064800.mseed
Saved segment file: HV_STCD_HHE_20241020T074800.mseed
Saved segment file: HV_STCD_HHE_20241020T084800.mseed
Saved segment file: HV_STCD_HHE_20241020T094800.mseed
Saved segment file: HV_STCD_HHE_20241020T104800.mseed
Saved segment file: HV_STCD_HHE_20241020T114800.mseed
Saved segment file: HV_STCD_HHE_20241020T124800.mseed
Saved segment file: HV_STCD_HHE_20241020T134800.mseed
Saved segment file: HV_STCD_HHE_20241020T144800.mseed
Saved segment file: HV_STCD_HHE_20241020T154800.mseed
Saved segment file: HV_STCD_

In [2]:
# check all files in the segmented folder for nan data

for file_path in glob.glob(f"{segmented_output_folder}/*.mseed"):
    st = read(file_path)
    for tr in st:
        if np.isnan(tr.data).any():
            print(f"File {file_path} contains NaN data")
        else:
            print(f"File {file_path} contains no NaN data")


File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20241012T084800.mseed contains no NaN data
File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20240914T144800.mseed contains no NaN data
File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20241003T134800.mseed contains no NaN data
File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20241027T224800.mseed contains no NaN data
File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20240929T054800.mseed contains no NaN data
File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20241007T144800.mseed contains no NaN data
File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20241029T184800.mseed contains no NaN data
File /home/jen/ESS/MLGEO202

File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20240928T204800.mseed contains no NaN data
File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20240908T014800.mseed contains no NaN data
File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20241014T044800.mseed contains no NaN data
File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20240923T094800.mseed contains no NaN data
File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20241015T094800.mseed contains no NaN data
File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20240904T154800.mseed contains no NaN data
File /home/jen/ESS/MLGEO2024_Geldingadalir/notebooks/HVSTCD_analysis/data/segmented/HV_STCD_HHE_20241009T154800.mseed contains no NaN data
File /home/jen/ESS/MLGEO202

Apply the Fourier Transform with the `specified window_length` and `hop_length parameters` to produce a spectrogram of size (96, 128) for each one-hour segment. The data is saved in `data\segmented`.

In [3]:
import os
import glob
import numpy as np
from obspy import read
import librosa
from scipy.signal import resample
import time
from IPython.display import display, clear_output
import torch

# Define directories
input_folder = os.getcwd() + '/data/raw'
processed_folder = os.getcwd() + '/data/processed'
segmented_output_folder = os.getcwd() + '/data/segmented'
os.makedirs(segmented_output_folder, exist_ok=True)  # Create output directory if it doesn't exist

# Define constants
target_sampling_rate = 8  # Hz
segment_duration = 3600  # 1 hour in seconds
window_length = 256
hop_length = 224
target_shape = (96, 128)

# Initialize an empty list to store spectrograms
spectrogram_list = []

# Step 1: Process each file in the input folder
for file_path in glob.glob(f"{input_folder}/*.mseed"):
    # Read the file
    st = read(file_path)
    
    # Process each trace
    for tr in st:
        # Resample the data to 8 Hz (assuming original sampling rate was 100 Hz)
        # this is done in data cleaning
        #tr.resample(target_sampling_rate)
        
        # Split into one-hour segments (28,800 samples each)
        start_time = tr.stats.starttime
        end_time = tr.stats.endtime
        segment_start = start_time

        while segment_start + segment_duration <= end_time:
            # Define the end time for the current segment
            segment_end = segment_start + segment_duration

            # Slice the trace to create a one-hour segment
            segment = tr.slice(starttime=segment_start, endtime=segment_end)
            
            ## NEW ##
            # Convert to PyTorch tensor
            segment_tensor = torch.tensor(segment, dtype=torch.float32)
            
            # Ensure correct dimensions (STFT expects 1D tensor or batch x time)
            if len(segment_tensor.shape) == 1:
                segment_tensor = segment_tensor.unsqueeze(0)
                
            # Compute the STFT
            stft_result = torch.stft(
                segment_tensor,
                n_fft=window_length,
                hop_length=hop_length,
                win_length=window_length,
                return_complex=True,
            )
                
            # Compute the spectrogram (magnitude squared)
            spectrogram = torch.abs(stft_result) ** 2

            # Convert to NumPy for further processing
            spectrogram = spectrogram.numpy()

            # Resize to a consistent shape if needed (example using interpolation)
            target_shape = (96, 128)  # Desired shape
            spectrogram_resized = np.resize(spectrogram, target_shape)

            # Log-transform the spectrogram
            spectrogram_log = np.log1p(spectrogram_resized)

            # Normalize between -1 and 1
            spectrogram_log = 2 * (spectrogram_log - np.min(spectrogram_log)) / np.ptp(spectrogram_log) - 1
            
            ## END NEW ##

            # Save the spectrogram (you could save the result as needed)
            segment_filename = f"{tr.stats.network}_{tr.stats.station}_{tr.stats.channel}_{segment_start.strftime('%Y%m%dT%H%M%S')}.npy"
            segment_filepath = os.path.join(segmented_output_folder, segment_filename)

            np.save(segment_filepath, spectrogram_log)

            # Append the spectrogram to the list
            spectrogram_list.append(spectrogram_log)
            
            # check the file for data outside the bounds of -1 and 1
            if np.any(spectrogram_log > 1) or np.any(spectrogram_log < -1):
                print(f"File {segment_filename} contains data outside the bounds of -1 and 1")

            # Move to the next hour
            segment_start = segment_end
            
            # display file being processed
            # time.sleep(0.1)
            # clear_output(wait=True)
            print(f'Processing file: {segment_filename}')
            
            #check file for nan data
            if np.isnan(spectrogram_log).any():
                print(f"File {segment_filename} contains NaN data")             

# Stack all spectrograms into a single numpy array and add batch and channel dimensions
all_spectrograms = np.stack(spectrogram_list)
all_spectrograms = all_spectrograms[:, np.newaxis, :, :]  # Shape: (batch_size, 1, 96, 128)
print(f"Shape of combined spectrogram array: {all_spectrograms.shape}")

# remove the single feature dimension (1)
all_spectrograms = np.squeeze(all_spectrograms)

# NOTE: FILE NAME IS HARD CODED HERE
# Save the combined spectrogram array to Input.npy
final_output_folder = os.getcwd()
input_filepath = os.path.join(final_output_folder, "Input.npy")
#input_filepath = os.path.join(final_output_folder + 'NUPH_analysis', "Input_nuph.npy")
np.save(input_filepath, all_spectrograms)

print(f"Saved combined spectrogram file: {input_filepath.split('/')[-1]}")


/home/jen/miniconda3/envs/mlgeo/lib/python3.9/site-packages/torch/functional.py:704: UserWarning: A window was not provided. A rectangular window will be applied,which is known to cause spectral leakage. Other windows such as torch.hann_window or torch.hamming_window can are recommended to reduce spectral leakage.To suppress this warning and use a rectangular window, explicitly set `window=torch.ones(n_fft, device=<device>)`. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1730833640211/work/aten/src/ATen/native/SpectralOps.cpp:836.)
  return _VF.stft(  # type: ignore[attr-defined]


Processing file: HV_STCD_HHE_20241019T224800.npy
Processing file: HV_STCD_HHE_20241019T234800.npy
Processing file: HV_STCD_HHE_20241020T004800.npy
Processing file: HV_STCD_HHE_20241020T014800.npy
Processing file: HV_STCD_HHE_20241020T024800.npy
Processing file: HV_STCD_HHE_20241020T034800.npy
Processing file: HV_STCD_HHE_20241020T044800.npy
Processing file: HV_STCD_HHE_20241020T054800.npy
Processing file: HV_STCD_HHE_20241020T064800.npy
Processing file: HV_STCD_HHE_20241020T074800.npy
Processing file: HV_STCD_HHE_20241020T084800.npy
Processing file: HV_STCD_HHE_20241020T094800.npy
Processing file: HV_STCD_HHE_20241020T104800.npy
Processing file: HV_STCD_HHE_20241020T114800.npy
Processing file: HV_STCD_HHE_20241020T124800.npy
Processing file: HV_STCD_HHE_20241020T134800.npy
Processing file: HV_STCD_HHE_20241020T144800.npy
Processing file: HV_STCD_HHE_20241020T154800.npy
Processing file: HV_STCD_HHE_20241020T164800.npy
Processing file: HV_STCD_HHE_20241020T174800.npy
Processing file: HV_

In [4]:
# import matplotlib.pyplot as plt

# # Ensure there are spectrograms to plot
# if all_spectrograms.size > 0:
#     length = len(all_spectrograms)
#     print(length)
#     plt.figure(figsize=(10, 6))
#     for i in range(length):  # Loop through all spectrograms
#         plt.plot(all_spectrograms[i].flatten())
#     plt.title(f'Spectrograms as Line Plot')
#     plt.xlabel('Frequency Bins')
#     plt.ylabel('Log Power')
#     plt.show()
# else:
#     print("No spectrograms found in all_spectrograms.")

In [5]:
# display information on Input.npy
print(f"Shape of combined spectrogram array with batch and channel dimensions: {all_spectrograms.shape}")
print(f"Number of files processed: {all_spectrograms.shape[0]}")

# load Input.npy and display information for confirmation
input_data_file_t = np.load(input_filepath)
print(f"Shape of combined spectrogram array Input.npy: {input_data_file_t.shape}")
print(f"Number of files processed: {input_data_file_t.shape[0]}")

# the numbers should match

Shape of combined spectrogram array with batch and channel dimensions: (1560, 96, 128)
Number of files processed: 1560
Shape of combined spectrogram array Input.npy: (1560, 96, 128)
Number of files processed: 1560


In [6]:
# display first five rows in the all_spectograms
print(all_spectrograms[:5])

[[[ 0.948272    0.7222569   0.4166124  ...  0.62695956  0.8384929
    0.5973412 ]
  [ 0.5167481   0.5086564   0.62962186 ...  0.69485915  0.32145786
    0.73210335]
  [ 0.7979194   0.8190564   0.882458   ...  0.71615195  0.21966577
    0.87362504]
  ...
  [ 0.32925415  0.3006282   0.2804972  ...  0.05321515 -0.20024502
   -0.06325483]
  [-0.32069606  0.01012921  0.04910898 ... -0.19394702 -0.41507137
    0.00877297]
  [ 0.21275938  0.22996879  0.0399195  ... -0.47082448  0.08446002
   -0.00821948]]

 [[ 0.6780112   0.7809503   0.74884987 ...  0.10246253  0.6796757
    0.15602124]
  [ 0.6396531   0.6804576   0.32985067 ...  0.02287841  0.8444772
    0.5740298 ]
  [ 0.37451792  0.8697138   0.7246758  ...  0.69237757 -0.03417015
    0.64687955]
  ...
  [-0.32579362  0.10342944  0.1271106  ... -0.11428934  0.03568149
    0.148162  ]
  [ 0.01707256 -0.0718953  -0.0056445  ...  0.17459416 -0.03449529
   -0.3287264 ]
  [-0.4357369   0.1211797   0.22760177 ... -0.56148386 -0.05048811
    0.096